In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import plotly.express as px

# Load dataset
file_path = "Data.csv"  # Ensure 'Data.csv' is in the same directory as this script
df = pd.read_csv(file_path)

# Convert Timestamp to datetime
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# Ensure PM2.5 values are numeric and clean data
df["PM2.5"] = pd.to_numeric(df["PM2.5"], errors="coerce")
df = df.dropna(subset=["PM2.5", "state", "Timestamp"])  

# Unique states list
states = sorted(df["state"].dropna().unique())

# Custom CSS Styling for UI (unchanged)
custom_css = """
<style>
    body { background-color: #f4f4f4; font-family: Arial, sans-serif; }
    .header { text-align: center; font-size: 26px; font-weight: bold; padding: 10px; color: #fff; background: linear-gradient(to right, #007BFF, #00C1FF); border-radius: 10px; }
    .container { display: flex; flex-direction: column; align-items: center; }
    .box { width: 60%; padding: 10px; text-align: center; }
    .button { width: 250px !important; height: 50px !important; font-size: 16px !important; border-radius: 20px !important; background: linear-gradient(to right, #ff416c, #ff4b2b); color: white !important; font-weight: bold; border: none !important; cursor: pointer; }
    .button:hover { background: linear-gradient(to right, #ff4b2b, #ff416c); }
    .dropdown, .date-picker { width: 300px !important; height: 40px !important; font-size: 14px !important; text-align: center !important; }
</style>
"""

# Create widgets
state_dropdown = widgets.Dropdown(
    options=states,
    description="State:",
    layout=widgets.Layout(width="300px")
)

date_picker_start = widgets.DatePicker(
    description="From:",
    layout=widgets.Layout(width="200px")
)

date_picker_end = widgets.DatePicker(
    description="To:",
    layout=widgets.Layout(width="200px")
)

apply_button = widgets.Button(
    description="Apply Filter",
    button_style="primary",
    layout=widgets.Layout(width="250px", height="50px"),
    style={"button_color": "#ff416c"}
)
# Add custom styling to the button using a custom CSS class
apply_button.add_class("button")

output = widgets.Output()

# Function to filter and plot data interactively with Plotly Express
def filter_and_plot(_):
    output.clear_output()
    with output:
        state_selected = state_dropdown.value
        start_date = date_picker_start.value
        end_date = date_picker_end.value

        # Check if both dates are selected
        if not start_date or not end_date:
            display(HTML("<p style='color:red; font-weight:bold;'>⚠️ Please select both dates.</p>"))
            return

        # Filter data
        filtered_df = df[(df["state"] == state_selected) & 
                         (df["Timestamp"] >= pd.Timestamp(start_date)) & 
                         (df["Timestamp"] <= pd.Timestamp(end_date))]

        if filtered_df.empty:
            display(HTML(f"<p style='color:red; font-weight:bold;'>⚠️ No data found for {state_selected} in the selected range.</p>"))
            return

        # Create interactive line plot with Plotly Express
        fig = px.line(
            filtered_df,
            x="Timestamp",
            y="PM2.5",
            title=f"PM2.5 Levels in {state_selected} ({start_date} to {end_date})",
            markers=True
        )

        # Apply some simple styling
        fig.update_layout(
            title_font=dict(size=18, family='Arial', color='#333'),
            xaxis_title="Date",
            yaxis_title="PM2.5",
            template="plotly_white",
            margin=dict(l=40, r=40, t=80, b=40)
        )
        fig.update_xaxes(rangeslider_visible=True, tickangle=45)

        # Display interactive figure
        fig.show()

# Bind function to button
apply_button.on_click(filter_and_plot)

# Display UI
display(HTML(custom_css))  # Apply custom CSS
display(HTML("<div class='header'>📊 PM2.5 Level Analysis Dashboard</div>"))  # Header
display(
    widgets.VBox([
        widgets.HBox([state_dropdown], layout=widgets.Layout(justify_content="center", padding="10px")),
        widgets.HBox([date_picker_start, date_picker_end], layout=widgets.Layout(justify_content="center", width="60%", padding="10px")),
        widgets.HBox([apply_button], layout=widgets.Layout(justify_content="center", padding="10px")),
        output
    ], layout=widgets.Layout(align_items="center"))
)


FileNotFoundError: [Errno 2] No such file or directory: 'Data.csv'